# 1. Вытаскиваем ссылки, которые накачали

In [ ]:
pip install fake_useragent

In [10]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError

# библиотека для создания юзерагентов
from fake_useragent import UserAgent

from tqdm import tqdm_notebook
import re
import pickle
import time
 
# модуль с мелкими функциями
from cian_info import *

In [11]:
with open('hrefs_total.pickle', 'rb') as f:
    hrefs_total = pickle.load(f)
    
len(hrefs_total)

73924

In [12]:
type(hrefs_total), len(set(hrefs_total))

(list, 73924)

### Чтобы включать ТОР

In [13]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [14]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 185.220.100.240


# 2. Качаем основную информацию по квартирам

Объединяем всё в одну функцию! 

In [15]:
def infa_download(soup):
    infa_dict = {}
    infa_dict.update(get_general_information(soup))
    infa_dict.update(get_about(soup))
    infa_dict.update(get_price(soup))
    infa_dict.update(get_description(soup))
    infa_dict.update(get_square_info(soup))
    infa_dict.update(get_address(soup))
    infa_dict.update(get_metro(soup))
    infa_dict.update(get_title(soup))
    infa_dict.update(get_coord(soup))
    infa_dict.update(get_reinovation(soup))
    return infa_dict

Основной цикл: 

In [16]:
all_infa = [ ] # для данных

In [ ]:
all_infa = data_new

In [ ]:
for url in tqdm_notebook(hrefs_total[30344:40000]):
    print(url)
    
    try:
        soup = get_soup(url)
        
    except ConnectionError:
        print('конэкшон ерор')
        time.sleep(5)
        soup = get_soup(url)
        
    # пока вылезает капча, отдыхаем
    try:
        while is_capcha_or_badip(soup):
            time.sleep(15)            # сплю N секунд
            soup = get_soup(url)     # внеочередная попытка пробится сквозь капчу 

        if soup.find("div", {"data-name": "OfferUnpublished"}):
            print('объявление снято')
            continue

        cur_infa = infa_download(soup)
        cur_infa['href'] = url
        all_infa.append(cur_infa)
    except AttributeError:
        continue

https://www.cian.ru/sale/flat/223825147/
Подозрительная сеть :(
объявление снято
https://www.cian.ru/sale/flat/227474330/
https://www.cian.ru/sale/flat/224531374/
https://www.cian.ru/sale/flat/226270567/
объявление снято
https://www.cian.ru/sale/flat/225133041/
объявление снято
https://www.cian.ru/sale/flat/226773649/
https://www.cian.ru/sale/flat/220090018/
Капча вылезла :(
Капча вылезла :(
Капча вылезла :(
Капча вылезла :(
https://www.cian.ru/sale/flat/224678577/
объявление снято
https://www.cian.ru/sale/flat/223154697/
https://www.cian.ru/sale/flat/213858552/
https://www.cian.ru/sale/flat/225061601/
объявление снято
https://www.cian.ru/sale/flat/209882556/
https://www.cian.ru/sale/flat/223652645/
https://www.cian.ru/sale/flat/222813925/
https://www.cian.ru/sale/flat/224349498/
объявление снято
https://www.cian.ru/sale/flat/221411048/
объявление снято
https://www.cian.ru/sale/flat/219121501/
https://www.cian.ru/sale/flat/226294903/
объявление снято
https://www.cian.ru/sale/flat/22634

https://www.cian.ru/sale/flat/225169951/
объявление снято
https://www.cian.ru/sale/flat/222547785/
https://www.cian.ru/sale/flat/214363759/
https://www.cian.ru/sale/flat/227512410/
объявление снято
https://www.cian.ru/sale/flat/227149336/
https://www.cian.ru/sale/flat/226923518/
объявление снято
https://www.cian.ru/sale/flat/227625792/
https://www.cian.ru/sale/flat/218935390/
объявление снято
https://www.cian.ru/sale/flat/225457316/
объявление снято
https://www.cian.ru/sale/flat/224628122/
https://www.cian.ru/sale/flat/225743732/
https://www.cian.ru/sale/flat/226694106/
объявление снято
https://www.cian.ru/sale/flat/202548165/
https://www.cian.ru/sale/flat/217207445/
https://www.cian.ru/sale/flat/226329152/
объявление снято
https://www.cian.ru/sale/flat/222420403/
объявление снято
https://www.cian.ru/sale/flat/225179130/
https://www.cian.ru/sale/flat/203798002/
https://www.cian.ru/sale/flat/218098824/
Капча вылезла :(
Капча вылезла :(
объявление снято
https://www.cian.ru/sale/flat/2271

https://www.cian.ru/sale/flat/225352054/
https://www.cian.ru/sale/flat/179166468/
Капча вылезла :(
Капча вылезла :(
https://www.cian.ru/sale/flat/226100716/
https://www.cian.ru/sale/flat/226392852/
https://www.cian.ru/sale/flat/225362143/
https://www.cian.ru/sale/flat/227233188/
https://www.cian.ru/sale/flat/206127781/
https://www.cian.ru/sale/flat/226254934/
https://www.cian.ru/sale/flat/222441135/
https://www.cian.ru/sale/flat/226994549/
объявление снято
https://www.cian.ru/sale/flat/225632478/
https://www.cian.ru/sale/flat/226717463/
https://www.cian.ru/sale/flat/181101740/
https://www.cian.ru/sale/flat/224883425/
объявление снято
https://www.cian.ru/sale/flat/219552480/
https://www.cian.ru/sale/flat/226547361/
объявление снято
https://www.cian.ru/sale/flat/161568723/
https://www.cian.ru/sale/flat/202605876/
объявление снято
https://www.cian.ru/sale/flat/224168842/
Капча вылезла :(
объявление снято
https://www.cian.ru/sale/flat/226329489/
объявление снято
https://www.cian.ru/sale/fl

https://www.cian.ru/sale/flat/218397463/
Подозрительная сеть :(
Капча вылезла :(
Капча вылезла :(
Подозрительная сеть :(
Подозрительная сеть :(
https://www.cian.ru/sale/flat/226854826/
объявление снято
https://www.cian.ru/sale/flat/226539793/
объявление снято
https://www.cian.ru/sale/flat/222597072/
объявление снято
https://www.cian.ru/sale/flat/221133244/
https://www.cian.ru/sale/flat/226212507/
объявление снято
https://www.cian.ru/sale/flat/225840551/
объявление снято
https://www.cian.ru/sale/flat/226166800/
объявление снято
https://www.cian.ru/sale/flat/227053372/
Капча вылезла :(
https://www.cian.ru/sale/flat/227061635/
https://www.cian.ru/sale/flat/227547340/
https://www.cian.ru/sale/flat/196794460/
https://www.cian.ru/sale/flat/219767673/
https://www.cian.ru/sale/flat/226317492/
https://www.cian.ru/sale/flat/226135222/
https://www.cian.ru/sale/flat/224340510/
объявление снято
https://www.cian.ru/sale/flat/227080992/
https://www.cian.ru/sale/flat/225854998/
Капча вылезла :(
Подозр

https://www.cian.ru/sale/flat/225061868/
https://www.cian.ru/sale/flat/226422856/
Капча вылезла :(
объявление снято
https://www.cian.ru/sale/flat/219788018/
объявление снято
https://www.cian.ru/sale/flat/227630315/
объявление снято
https://www.cian.ru/sale/flat/227708789/
https://www.cian.ru/sale/flat/198373089/
Капча вылезла :(
Капча вылезла :(
объявление снято
https://www.cian.ru/sale/flat/226708525/
объявление снято
https://www.cian.ru/sale/flat/227065225/
https://www.cian.ru/sale/flat/221469402/
объявление снято
https://www.cian.ru/sale/flat/211015687/
Капча вылезла :(
Капча вылезла :(
Капча вылезла :(
Капча вылезла :(
https://www.cian.ru/sale/flat/227636319/
объявление снято
https://www.cian.ru/sale/flat/217758591/
https://www.cian.ru/sale/flat/223421976/
https://www.cian.ru/sale/flat/221692636/
объявление снято
https://www.cian.ru/sale/flat/199596585/
объявление снято
https://www.cian.ru/sale/flat/223984563/
Капча вылезла :(
https://www.cian.ru/sale/flat/227179290/
объявление сня

объявление снято
https://www.cian.ru/sale/flat/218477654/
объявление снято
https://www.cian.ru/sale/flat/225281308/
https://www.cian.ru/sale/flat/227250913/
https://www.cian.ru/sale/flat/224855475/
Капча вылезла :(
https://www.cian.ru/sale/flat/226372175/
объявление снято
https://www.cian.ru/sale/flat/223753935/
объявление снято
https://www.cian.ru/sale/flat/215296252/
объявление снято
https://www.cian.ru/sale/flat/223510024/
объявление снято
https://www.cian.ru/sale/flat/226415842/
https://www.cian.ru/sale/flat/227566540/
https://www.cian.ru/sale/flat/221454710/
https://www.cian.ru/sale/flat/217505180/
https://www.cian.ru/sale/flat/204842640/
https://www.cian.ru/sale/flat/226507906/
объявление снято
https://www.cian.ru/sale/flat/227640690/
объявление снято
https://www.cian.ru/sale/flat/222392931/
https://www.cian.ru/sale/flat/227577636/
https://www.cian.ru/sale/flat/227577537/
https://www.cian.ru/sale/flat/149558497/
https://www.cian.ru/sale/flat/226203956/
https://www.cian.ru/sale/fl

https://www.cian.ru/sale/flat/221450599/
объявление снято
https://www.cian.ru/sale/flat/206096162/
https://www.cian.ru/sale/flat/227001188/
https://www.cian.ru/sale/flat/219039982/
https://www.cian.ru/sale/flat/225080934/
объявление снято
https://www.cian.ru/sale/flat/226475967/
объявление снято
https://www.cian.ru/sale/flat/221763041/
https://www.cian.ru/sale/flat/225245313/
https://www.cian.ru/sale/flat/226942570/
объявление снято
https://www.cian.ru/sale/flat/226364583/
объявление снято
https://www.cian.ru/sale/flat/225279576/
https://www.cian.ru/sale/flat/222212224/
https://www.cian.ru/sale/flat/225365514/
Капча вылезла :(
объявление снято
https://www.cian.ru/sale/flat/194592864/
объявление снято
https://www.cian.ru/sale/flat/227732867/
https://www.cian.ru/sale/flat/227001918/
https://www.cian.ru/sale/flat/227478052/
https://www.cian.ru/sale/flat/210829093/
объявление снято
https://www.cian.ru/sale/flat/208869202/
https://www.cian.ru/sale/flat/226845340/
объявление снято
https://ww

https://www.cian.ru/sale/flat/224184542/
объявление снято
https://www.cian.ru/sale/flat/225078654/
https://www.cian.ru/sale/flat/227709431/
https://www.cian.ru/sale/flat/227358420/
https://www.cian.ru/sale/flat/217696687/
https://www.cian.ru/sale/flat/226671023/
Капча вылезла :(
Капча вылезла :(
Подозрительная сеть :(
Капча вылезла :(
Подозрительная сеть :(
Капча вылезла :(
объявление снято
https://www.cian.ru/sale/flat/221890396/
https://www.cian.ru/sale/flat/214598073/
https://www.cian.ru/sale/flat/216766169/
https://www.cian.ru/sale/flat/222809467/
Капча вылезла :(
Капча вылезла :(
Капча вылезла :(
https://www.cian.ru/sale/flat/224042251/
https://www.cian.ru/sale/flat/225803667/
объявление снято
https://www.cian.ru/sale/flat/213662100/
https://www.cian.ru/sale/flat/226705763/
объявление снято
https://www.cian.ru/sale/flat/225868713/
объявление снято
https://www.cian.ru/sale/flat/222646283/
объявление снято
https://www.cian.ru/sale/flat/225524940/
https://www.cian.ru/sale/flat/221447

In [20]:
hrefs_total.index(url) 
# чтобы понимать откуда продолжать скачку, если код упал

30344

In [21]:
len(all_infa)

180

In [ ]:
# осторожнее с именами файлов! случайно можно перезаписать готовый :( 
with open('data/info_30000_40000.pickle', 'wb') as f:
    pickle.dump(all_infa, f)

In [8]:
with open('data/info_30000_40000.pickle', 'rb') as f:
    data_new = pickle.load(f)

EOFError: Ran out of input

Поглазеть на данные: 

In [ ]:
import pandas as pd
df = pd.DataFrame(data_new)
df.tail()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()